In [5]:
import tensorflow as tf


In [6]:
import numpy as np


In [7]:
from typing import List,Dict
def debug(func):
    def r(*args,**kwargs):
        print(locals())
        return func(*args,**kwargs)
    return r


In [8]:
var=tf.Variable
size=lambda x:x.shape[1:3].as_list()
channal=lambda x:int(x.shape[3])
count=lambda x:int(x.shape[0])

In [9]:
@debug
def getFileters(h,w,inc,outc):
    return var(np.random.uniform(size=(h,w,inc,outc)),dtype="float32")

In [10]:
def conv(h,w,c,stride=1):            
    def run(tensor):
        f=getFileters(h,w,channal(tensor),c)
        x=tf.nn.conv2d(input=tensor,filter=f,strides=[1,stride,stride,1],padding="SAME")
        return x
    return run

# nested function 
# conv(hwcs)(tensor)()
# repeat conv can create 

In [11]:
def combine(layers:list):
    """
    convs:list of function , function here is generated by conv function
    standard:the output of previous one should be 
    """
    def run(tensor):
        now=tensor
        for func in layers:
            now=func(now)
        return now
    return run

In [12]:
def shortcut(seq):
    """
    cross-layer connections
    seq there is defined as a function that is just like "func(input)->output"
    the input and output of seq should 
    """
    def run(tensor):
        x=seq(tensor)
        return tensor+x
    return run


In [13]:
def transtoResblock(convs):
    """
    user function
    generate resblock
    """
    s=combine(convs)
    return shortcut(s)

In [14]:
# define the conv function as function(h,w,c)->func(tensor)->output
def shrinkConv(h,w,c):
    """
    using padding to ensures that the size is halved
    """
    return conv(h,w,c,stride=2)

In [15]:
def getResblock(listoftriple):
    """
    user function
    triple is defined as (h,w,c)
    """
    # map triple to conv and generate a resblock
    convs=list(map(lambda triple:conv(*triple),listoftriple))
    res=transtoResblock(convs)
    return res


In [16]:
# new starting point
def getResList(countOfConvs,inputshape):
    """
    map [starttriple,endstriple] -> [starttriple,middletriple......,endstriple]
    auto generate middle layer
    """
    assert countOfConvs%2==0,"count must be even"
    # temporarily adopt the strategy of  "single shrinkage middle layer" 
    # randomly generate a list of triples
    c=inputshape[3]
    def changeChannal(c):
        ret=list(inputshape)
        ret[3]=c
        return ret
    # even corresponds to half
    # odd corresponds to all
    lst=[
        changeChannal(c//2)[1:],
        changeChannal(c)[1:]
    ]
    def randomTriple(idx):
        triple=lst[idx%2]
        return triple
            
    ret=list(map(lambda x:randomTriple(x),range(countOfConvs)))
    return ret
    

In [17]:
def yolo_resblock():
    def run(tensor):
        lst=getResList(countOfConvs=2,inputshape=tensor.shape.as_list())
        resb=getResblock(lst)
        return resb(tensor)
    return run

In [18]:
lst=getResList(countOfConvs=2,inputshape=(5,5,5,5))

In [19]:
testfunc=getResblock(lst)

In [20]:
testtensor=tf.constant(np.random.uniform(size=(5,400,400,5)),dtype="float32")

In [21]:
sess=tf.Session()


In [22]:
# here is the tensorboard section
from tensorflow.summary import FileWriter
writer=FileWriter("./logs",sess.graph)



In [23]:
inputsize=416
inputch=3


In [24]:
x=yolo_resblock()(testtensor)
# x=yolo_resblock()(x)
# x=yolo_resblock()(x)
print(x)

{'args': (400, 400, 5, 2), 'kwargs': {}, 'func': <function getFileters at 0x00000224F9BC7678>}
{'args': (400, 400, 2, 5), 'kwargs': {}, 'func': <function getFileters at 0x00000224F9BC7678>}
Tensor("add:0", shape=(5, 400, 400, 5), dtype=float32)


In [21]:
initop=tf.global_variables_initializer()
sess.run(initop)

In [22]:

sess.run(x)

In [1]:
v=getFileters(5,5,channal(testtensor),5)
t=tf.nn.conv2d(testtensor,filter=v,padding="SAME")
sess.run(tf.global_variables_initializer())
print(t)

NameError: name 'getFilete' is not defined

In [2]:
sess.run(t)

NameError: name 'sess' is not defined

In [3]:
tf.


SyntaxError: invalid syntax (<ipython-input-3-152b50a4a1ee>, line 1)